In [93]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [94]:
import tensorflow as tf
from tensorflow.keras import layers, preprocessing, utils
import yaml

print( tf.version)

<module 'tensorflow._api.v2.version' from '/home/samroadie/.local/lib/python3.8/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [95]:
import os

In [96]:
os.listdir()

['Untitled.ipynb',
 'data',
 'download1.png',
 'vaniquery.ipynb',
 'seq2seq-chatbot-keras-with-attention.ipynb',
 'model.h5',
 'MyModel_tf',
 'datac',
 'gn vs epoch.png',
 'home',
 'gn vs epoch scatter.png',
 'glove.6B.50d.txt',
 'InfoBot.ipynb',
 'model.pth',
 'archive.zip',
 'download2.png',
 '.ipynb_checkpoints',
 'acing_pytorch.ipynb',
 'loadedmodel.h5',
 'InfoBot(update).ipynb',
 'from-SCRATCH',
 'Untitled1.ipynb']

In [97]:
dirname = 'datac'

In [98]:
questions=list()
answers=list()

for dirname, _, filenames in os.walk('/home/samroadie/Desktop/ace_pytorch/datac'):
    for filename in filenames:
        print(filename)
        file = open(os.path.join(dirname, filename), 'rb')
        docs = yaml.safe_load(file)
        conversations = docs['conversations']
        for con in conversations:
            if len(con)>2:
                questions.append(con[0])
                replies=con[1:]
                ans=""
                for rep in replies:
                    ans+=' '+rep
                answers.append(ans)
            elif len(con)>1:
                questions.append(con[0])
                answers.append(con[1])

answers_with_tags = list()
for i in range( len(answers)):
    if type(answers[i])==str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers=list()

for ans in answers_with_tags:
    answers.append('<START> '+ans+' <END>')

tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions+answers)
VOCAB_SIZE= len(tokenizer.word_index)+1
print('VOCAB SIZE : {}'.format(VOCAB_SIZE))

sports.yml
emotion.yml
ai.yml
botprofile.yml
politics.yml
food.yml
science.yml
computers.yml
psychology.yml
history.yml
movies.yml
literature.yml
trivia.yml
money.yml
health.yml
gossip.yml
humor.yml
greetings.yml
VOCAB SIZE : 1894


In [99]:
answers_with_tags

['The Gold Glove.',
 'Snowboarding.',
 'A game with tall players.',
 'I was born without the sports gene.',
 'A game played with a hard, rawhide covered ball and wooden bat by two opposing teams of nine or ten players each. It is played on a field with four bases forming a diamond-shaped circuit.',
 'A game played with a round ball by two teams of eleven players on a field with a goal at either end; the ball is moved chiefly by kicking or by using any part of the body except the hands and arms.',
 'I am not into sports that much.',
 'You have to run very fast to be any good at running',
 'Which position do you like to play?',
 'Cricket is a bat-and-ball game played between two teams of eleven players on a cricket field, at the centre of which is a rectangular 22-yard-long pitch with a wicket (a set of three wooden stumps) sited at each end.',
 'Does take up a lot of your time?',
 "I don't know how to play",
 "No I don't have the coordination for hoops.",
 ' WHAT IS BASKETBALL? DO YOU W

In [100]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [101]:
questions

['EACH YEAR IN PRO BASEBALL THE ',
 'IF YOU ARE RIDING FAKIE INSIDE',
 'WHAT IS BASKETBALL',
 'WHAT SOCCER',
 'WHAT IS BASEBALL',
 'WHAT IS SOCCER',
 'I LOVE BASEBALL',
 'I PLAY SOCCER',
 'I PLAY Cricket',
 'What is cricket',
 'I PLAY VOLLEYBALL',
 'DO YOU PLAY SOCCER',
 'DO YOU PLAY BASKETBALL',
 'DO YOU KNOW BASKETBAL',
 'LIKE BASKETBALL',
 'ARE YOU A FOOTBALL',
 'WHO IS THE GREATEST BASEBALL PLAYER',
 'WHO IS THE BEST SOCCER PLAYER',
 'TELL ME ABOUT BASEBALL',
 'Which is your favourite soccer club?',
 'You are arrogant',
 'You are bragging',
 'You are never sad',
 'You are jealous',
 'You are never nice',
 'You will be happy',
 'You should be ashamed',
 'You can not feel',
 'You can not experience',
 'Have you felt',
 'Have you ever love',
 'Does that make you',
 'Does it make you sad',
 'Feelings',
 'What is your fear',
 'What is your mood',
 'What makes you sad',
 'What makes you unhappy',
 'What makes you mad',
 'What do you worry',
 'What do you hate',
 'I have emotions',
 'I am

In [102]:
answers = answers_with_tags

In [103]:
len(answers)

564

In [104]:

sorted_ques = []
sorted_ans = []
for i in range(len(questions)):
    if len(questions[i]) < 30:
        sorted_ques.append(questions[i])
        sorted_ans.append(answers[i])



###############################
#                             #
###############################
import re



def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r"i'm", "i am", txt)
    txt = re.sub(r"he's", "he is", txt)
    txt = re.sub(r"she's", "she is", txt)
    txt = re.sub(r"that's", "that is", txt)
    txt = re.sub(r"what's", "what is", txt)
    txt = re.sub(r"where's", "where is", txt)
    txt = re.sub(r"\'ll", " will", txt)
    txt = re.sub(r"\'ve", " have", txt)
    txt = re.sub(r"\'re", " are", txt)
    txt = re.sub(r"\'d", " would", txt)
    txt = re.sub(r"won't", "will not", txt)
    txt = re.sub(r"can't", "can not", txt)
    txt = re.sub(r"[^\w\s]", "", txt)
    return txt

clean_ques = []
clean_ans = []

for line in sorted_ques:
    clean_ques.append(clean_text(line))
        
for line in sorted_ans:
    clean_ans.append(clean_text(line))



## delete
#del(answers, questions, line)


###############################
#                             #
###############################




In [105]:
len(clean_ques)

521

In [106]:
clean_ans

['a game with tall players',
 'i was born without the sports gene',
 'a game played with a hard rawhide covered ball and wooden bat by two opposing teams of nine or ten players each it is played on a field with four bases forming a diamondshaped circuit',
 'a game played with a round ball by two teams of eleven players on a field with a goal at either end the ball is moved chiefly by kicking or by using any part of the body except the hands and arms',
 'i am not into sports that much',
 'you have to run very fast to be any good at running',
 'which position do you like to play',
 'cricket is a batandball game played between two teams of eleven players on a cricket field at the centre of which is a rectangular 22yardlong pitch with a wicket a set of three wooden stumps sited at each end',
 'does take up a lot of your time',
 'i dont know how to play',
 'no i dont have the coordination for hoops',
 ' what is basketball do you want to play basketball i am all net baby',
 'i am into the ne

In [107]:
for i in range(len(clean_ans)):
    clean_ans[i] = ' '.join(clean_ans[i].split()[:20])



###############################
#                             #
###############################

#del(sorted_ans, sorted_ques)


## trimming
#clean_ans=clean_ans[:30000]
#clean_ques=clean_ques[:30000]
## delete


###  count occurences ###
word2count = {}

for line in clean_ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in clean_ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

## delete
#del(word, line)


###  remove less frequent ###
thresh = 3

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1
        
## delete
#del(word2count, word, count, thresh)       
#del(word_num)        



for i in range(len(clean_ans)):
    clean_ans[i] = '<SOS> ' + clean_ans[i] + ' <EOS>'



tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1
    
    

vocab['what'] = vocab['<PAD>']
vocab['<PAD>'] = 0

## delete
#del(token, tokens) 
#del(x)

### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}



## delete
#del(i)



encoder_inp = []
for line in clean_ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in clean_ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

### delete
#del(clean_ans, clean_ques, line, lst, word)

In [108]:
encoder_inp

[[373, 1, 2],
 [373, 3],
 [373, 1, 4],
 [373, 1, 3],
 [5, 6, 4],
 [5, 7, 3],
 [5, 7, 8],
 [373, 1, 8],
 [5, 7, 375],
 [9, 10, 7, 3],
 [9, 10, 7, 2],
 [9, 10, 11, 375],
 [12, 2],
 [13, 10, 14, 375],
 [15, 1, 16, 17, 3, 375],
 [18, 19, 20, 4],
 [10, 13, 375],
 [10, 13, 375],
 [10, 13, 21, 22],
 [10, 13, 23],
 [10, 13, 21, 24],
 [10, 25, 26, 27],
 [10, 28, 26, 29],
 [10, 30, 31, 32],
 [10, 30, 31, 33],
 [34, 10, 375],
 [34, 10, 35, 6],
 [36, 37, 38, 10],
 [36, 39, 38, 10, 22],
 [40],
 [373, 1, 41, 42],
 [373, 1, 41, 375],
 [373, 43, 10, 22],
 [373, 43, 10, 375],
 [373, 43, 10, 44],
 [373, 9, 10, 45],
 [373, 9, 10, 46],
 [5, 34, 47],
 [5, 48, 375],
 [49, 50],
 [51, 52],
 [51, 30, 5, 375, 10],
 [9, 31, 45],
 [9, 31, 53],
 [9, 10, 32, 54],
 [9, 10, 32, 47],
 [9, 10, 32, 375],
 [9, 10, 35, 55, 44],
 [9, 10, 35, 55, 375],
 [9, 10, 35, 55, 56],
 [9, 10, 35, 55, 52],
 [9, 10, 46, 57],
 [9, 10, 55, 375],
 [9, 10, 55, 44],
 [58, 39, 1, 31],
 [18, 19, 20, 59],
 [18, 19, 20, 41, 375],
 [13, 10, 29],

In [109]:
vocab

{'what': 373,
 'is': 1,
 'basketball': 2,
 'soccer': 3,
 'baseball': 4,
 'i': 5,
 'love': 6,
 'play': 7,
 'cricket': 8,
 'do': 9,
 'you': 10,
 'know': 11,
 'like': 12,
 'are': 13,
 'a': 14,
 'who': 15,
 'the': 16,
 'best': 17,
 'tell': 18,
 'me': 19,
 'about': 20,
 'never': 21,
 'sad': 22,
 'jealous': 23,
 'nice': 24,
 'will': 25,
 'be': 26,
 'happy': 27,
 'should': 28,
 'ashamed': 29,
 'can': 30,
 'not': 31,
 'feel': 32,
 'experience': 33,
 'have': 34,
 'ever': 35,
 'does': 36,
 'that': 37,
 'make': 38,
 'it': 39,
 'feelings': 40,
 'your': 41,
 'fear': 42,
 'makes': 43,
 'mad': 44,
 'worry': 45,
 'hate': 46,
 'emotions': 47,
 'am': 48,
 'something': 49,
 'fun': 50,
 'how': 51,
 'angry': 52,
 'lie': 53,
 'scared': 54,
 'get': 55,
 'bored': 56,
 'anyone': 57,
 'no': 58,
 'relationships': 59,
 'feeling': 60,
 'ai': 61,
 'sentient': 62,
 'sapient': 63,
 'sound': 64,
 'data': 65,
 'immortal': 66,
 'making': 67,
 'sense': 68,
 'any': 69,
 'move': 70,
 'over': 71,
 'robots': 72,
 'laugh': 73

In [110]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_inp = pad_sequences(encoder_inp, 30, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 30, padding='post', truncating='post')




decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 30, padding='post', truncating='post')


del(i)


In [111]:
VOCAB_SIZE = len(vocab)
MAX_LEN = 30

print(decoder_final_output.shape, decoder_inp.shape, encoder_inp.shape, len(vocab), len(inv_vocab), inv_vocab[0])

(521, 30) (521, 30) (521, 30) 377 377 <PAD>


In [112]:
from tensorflow.keras.utils import to_categorical
decoder_final_output = to_categorical(decoder_final_output, len(vocab))
decoder_final_output.shape

(521, 30, 377)

In [113]:
embeddings_index = {}
with open('glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loded!")

Glove Loded!


In [114]:

embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = embedding_matrix_creater(50, word_index=vocab)    

In [115]:
embedding_matrix.shape

(378, 50)

In [116]:
embedding_matrix[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [117]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional, Concatenate, Dropout, Attention

In [118]:
embed = Embedding(VOCAB_SIZE+1, 
                  50, 
                  
                  input_length=13,
                  trainable=True)

embed.build((None,))
embed.set_weights([embedding_matrix])

In [119]:
enc_inp = Input(shape=(30, ))

In [120]:
#embed = Embedding(VOCAB_SIZE+1, 50, mask_zero=True, input_length=13)(enc_inp)
enc_embed = embed(enc_inp)
enc_lstm = Bidirectional(LSTM(400, return_state=True, dropout=0.05, return_sequences = True))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = enc_lstm(enc_embed)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

enc_states = [state_h, state_c]


dec_inp = Input(shape=(30, ))
dec_embed = embed(dec_inp)
dec_lstm = LSTM(400*2, return_state=True, return_sequences=True, dropout=0.05)
output, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

# attention
attn_layer = AttentionLayer()
attn_op, attn_state = attn_layer([encoder_outputs, output])
decoder_concat_input = Concatenate(axis=-1)([output, attn_op])


dec_dense = Dense(VOCAB_SIZE, activation='softmax')
final_output = dec_dense(decoder_concat_input)

model = Model([enc_inp, dec_inp], final_output)

model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 50)       18900       input_4[0][0]   

In [121]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [122]:
model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=50, batch_size=24, validation_split=0.15)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
19/19 [==============================] - ETA: 0s - loss: 2.6393 - acc: 0.6078WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7ff568535d30> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
19/19 [==============================] - 12s 631ms/step - loss: 2.6393 - acc: 0.6078 - val_loss: 2.1484 - val_acc: 0.6283
Epoch 2/50
19/19 [==============================] - 10s 535ms/step - loss: 1.8006 - acc: 0.6683 - val_loss: 1.9007 - val_acc: 0.6502
Epoch 3/50
19/19 [==============================] - 11s 604ms/step - loss: 1.6229 - acc: 0.6897 - val_loss: 1.8491 - val_acc: 0.6616
Epoch 4/50
19/19 [==============================] - 11s 602ms/step - loss: 1.5367 - acc: 0.7011 - val_loss: 1.7726 - val_acc: 0.6810
Epoch 5/50
19/19 [==============================] - 11s 600ms/step - loss: 1.4629 - acc: 0.7093 - val_loss: 1.7755 - val_acc: 0.6865
Epoch 6/50
19/19 [=========================

In [123]:
enc_model = tf.keras.models.Model(enc_inp, [encoder_outputs, enc_states])


decoder_state_input_h = tf.keras.layers.Input(shape=( 400 * 2,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 400 * 2,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]

#decoder_output = dec_dense(decoder_outputs)

dec_model = tf.keras.models.Model([dec_inp, decoder_states_inputs],
                                      [decoder_outputs] + decoder_states)


In [125]:

from keras.preprocessing.sequence import pad_sequences
print("##########################################")
print("#       start chatting ver. 1.0          #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    
    prepro1 = input("you : ")
    try:
        prepro1 = clean_text(prepro1)
        prepro = [prepro1]
        
        txt = []
        for x in prepro:
            lst = []
            for y in x.split():
                try:
                    lst.append(vocab[y])
                except:
                    lst.append(vocab['<OUT>'])
            txt.append(lst)
        txt = pad_sequences(txt, 30, padding='post')


        ###
        enc_op, stat = enc_model.predict( txt )

        empty_target_seq = np.zeros( ( 1 , 1) )
        empty_target_seq[0, 0] = vocab['<SOS>']
        stop_condition = False
        decoded_translation = ''


        while not stop_condition :

            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + stat )

            ###
            ###########################
            attn_op, attn_state = attn_layer([enc_op, dec_outputs])
            decoder_concat_input = Concatenate(axis=-1)([dec_outputs, attn_op])
            decoder_concat_input = dec_dense(decoder_concat_input)
            ###########################

            sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )

            sampled_word = inv_vocab[sampled_word_index] + ' '

            if sampled_word != '<EOS> ':
                decoded_translation += sampled_word           


            if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            stat = [ h , c ] 

        print("chatbot attention : ", decoded_translation )
        print("==============================================")

    except:
        print("sorry didn't got you , please type again :( ")



##########################################
#       start chatting ver. 1.0          #
##########################################
you : what is ai
chatbot attention :  artificial intelligence is the <OUT> of <OUT> and science <OUT> to <OUT> <OUT> that 
you : who is my gf
chatbot attention :  a <OUT> 
you : what is basketball
chatbot attention :  a game with <OUT> players 
you : q
chatbot attention :  <OUT> is really <OUT> what is your favorite book by him 
